# Libraries Import


In [ ]:
# Common Libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

from glob import glob
from tqdm.auto import tqdm
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Model Libraries
import tensorflow as tf
tf.keras.backend.set_floatx('float32')

from keras import Model
from keras.models import Sequential
from keras.layers import Dense, Input, Embedding, Reshape, Concatenate
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam

# Get Data

- global commodity trade statistics data from kaggle(https://www.kaggle.com/datasets/unitednations/global-commodity-trade-statistics)
- longitudinal dataset


In [ ]:
gc_data = pd.read_csv("../gct_dataset/commodity_trade_statistics_data.csv")
gc_data.tail(3)

## Extract Data - Only Australia

- lmmnn example 참조


In [ ]:
coun_filter = ["Australia"]
au_data = gc_data[gc_data["country_or_area"].isin(coun_filter)]
au_data.reset_index(inplace=True, drop=True)
au_data.tail(3)

In [ ]:
comm_filter = ["Activated carbon", "Alarm clocks, non-electric", "Alcoholic liqueurs nes","Almonds in shell fresh or dried","Aluminous cement"]
au_import = au_data[au_data["commodity"].isin(comm_filter)]
au_import.reset_index(inplace=True,drop=True)
au_import.tail(3)

## Get Extra Dataset

- lmmnn 저자의 전처리 방법 구현
- 2가지 데이터(wheat, population)는 미적용


In [ ]:
cm_data = pd.read_csv("../gct_dataset/child-mortality-around-the-world.csv")
co_data = pd.read_csv("../gct_dataset/co-emissions-per-capita.csv")
dc_data = pd.read_csv("../gct_dataset/deaths-conflict-terrorism-per-100000.csv")
tm_data = pd.read_csv("../gct_dataset/hadcrut-surface-temperature-anomaly.csv")

In [ ]:
tm_data = tm_data[tm_data["Entity"].isin(coun_filter)]
tm_data = tm_data[(tm_data["Year"] >= 1988) & (tm_data["Year"] <= 2016)]
tm_data = tm_data.rename(columns={'Entity':'country_or_area',
                                  'Year':'year',
                                  'Surface temperature anomaly': 'temp_anomaly'})
tm_data.drop("Code",axis=1,inplace=True)
tm_data.reset_index(inplace=True,drop=True)
tm_data.tail(3)

In [ ]:
cm_data = cm_data[cm_data["Entity"].isin(coun_filter)]
cm_data = cm_data[(cm_data["Year"] >= 1988) & (cm_data["Year"] <= 2016)]
cm_data = cm_data.rename(columns={'Entity':'country_or_area',
                                  'Year':'year',
                                  'Child mortality rate - Sex: all - Age: 0-4 - Variant: estimates': 'child_mortality'})
cm_data.drop("Code",axis=1,inplace=True)
cm_data.reset_index(inplace=True,drop=True)
cm_data.tail(3)

In [ ]:
co_data = co_data[co_data["Entity"].isin(coun_filter)]
co_data = co_data[(co_data["Year"] >= 1988) & (co_data["Year"] <= 2016)]
co_data = co_data.rename(columns={'Entity':'country_or_area',
                                  'Year':'year',
                                  'Annual CO₂ emissions (per capita)': 'co2_emission'})
co_data.drop("Code",axis=1,inplace=True)
co_data.reset_index(inplace=True,drop=True)
co_data.tail(3)

In [ ]:
dc_data = dc_data[dc_data["Entity"].isin(coun_filter)]
dc_data = dc_data[(dc_data["Year"] >= 1988) & (dc_data["Year"] <= 2016)]
dc_data = dc_data.rename(columns={'Entity':'country_or_area',
                                  'Year':'year',
                                  'Deaths - Conflict and terrorism - Sex: Both - Age: All Ages (Rate)': 'death_conflict'})
dc_data.drop("Code",axis=1,inplace=True)
dc_data.reset_index(inplace=True,drop=True)
dc_data.tail(3)

In [ ]:
au_import = pd.merge(au_import, tm_data, on=['country_or_area', 'year'], how='left')
au_import = pd.merge(au_import, cm_data, on=['country_or_area', 'year'], how='left')
au_import = pd.merge(au_import, co_data, on=['country_or_area', 'year'], how='left')
au_import = pd.merge(au_import, dc_data, on=['country_or_area', 'year'], how='left')


In [ ]:
au_import.fillna(0, inplace=True)
au_import.tail(3)

In [ ]:
distinct_df = (
    au_import.drop_duplicates(subset='comm_code')
    .assign(commodity_id=lambda x: x.groupby('comm_code').ngroup())
)
distinct_df = distinct_df[["comm_code","commodity_id"]]

# inner join 수행
au_import = au_import.merge(distinct_df, on='comm_code')

# t 계산
au_import['t'] = (au_import['year'] - au_import['year'].min()) / (au_import['year'].max() - au_import['year'].min())

au_import.tail(3)

In [ ]:
cols_to_drop = ['commodity', 'year', 'comm_code']
au_import.drop(cols_to_drop, axis=1, inplace=True)
print(au_import.shape)
au_import.tail(5)

In [ ]:
## Label Encoding
nunique = au_import.nunique()
types = au_import.dtypes

categorical_columns = []
categorical_dims =  {}
for col in au_import.columns:
    print(col, au_import[col].nunique())
    if types[col] == 'object' or nunique[col] < 10:
        l_enc = LabelEncoder()
        au_import[col] = l_enc.fit_transform(au_import[col].values)
        categorical_columns.append(col)
        categorical_dims[col] = len(l_enc.classes_)

In [ ]:
au_import.tail(3)

In [ ]:
au_import['trade_usd'] = np.log(au_import['trade_usd'])
au_import['trade_usd'].plot(kind='hist', bins=20)
plt.show()

In [ ]:
au_import.rename(columns={'commodity_id': 'z0', 'trade_usd': 'y'}, inplace=True)

# Define MeNet


### DNN Model (Γ(Xi))


In [ ]:
# Menet에 사용하고자하는 DNN 모형 정의
def get_model():
    model = Sequential()
    model.add(Dense(units=32, activation='relu', input_dim=12))
    model.add(Dense(units=16, activation='relu'))
    model.add(Dense(units=1, activation='linear'))
    adam = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay= 0.0001, amsgrad=False)
    model.compile(loss='mse', optimizer=adam)
    return model

In [ ]:
# Featue Map 추출 하는 함수 정의
# DNN 모형의 출력 전 단계의 layer를 추출하여 X의 Feature map을 생성
def get_feature_map(model, X):
    last_layer = Model(inputs = model.input, outputs = model.layers[-2].output)
    return last_layer.predict(X, verbose=0)

### Negative Log Likelihood


#### Paper(MeNets)

$$ F(g,β,u|X,y)= Σ*{i=1}^{N}[(y_i −Γ(X_i)β−Γ(X_i)u_i)^T Σ*{\epsilon*i=1}^{-1} (y_i −Γ(X_i)β−Γ(X_i)u_i)+u_i^TΣ*{u}^{-1}u_i +log|Σ\epsilon_i|+log|Σu|]$$


#### Paper(LMMNN)
$$ NLL(f,g,{\theta}|y)=1/2(y−{f(X)})^{′}{V(g,{\theta})^{−1}}(y−{f(X)})+1/2log|V(g,{\theta})|+n/2log2{\pi} $$
$$ V{(g,{\theta})} = g(Z)D({\psi})g(Z)^{′} + {\sigma}^{2}_{e}I_{n} $$
$$ yˆ_{te} = {fˆ}(X_{te} + {gˆ}(Z_{te}){bˆ}) $$
$$ {bˆ} = D({\psi}ˆ){gˆ}(Z_{tr})^{′}V({gˆ},{{\theta}ˆ})^{-1}(y_{tr} - {fˆ}(X_{tr})) $$
$$ V({\theta}) = ZD(\psi)Z^{′} + {\sigma}^{2}_{e}I_{n} = {\Sigma}^{K-1}_{l=0}{\Sigma}^{K-1}_{m=0}Z_{l}D_{l,m}Z^{′}_{m} + {\sigma}^{2}_{e}I_{n} $$

#### Code
$$ nll_i = (y_i - f̂_i - Z_i * b̂_i)^T * inv(R̂_i) * (y_i - f̂_i - Z_i * b̂_i) + b̂_i^T * inv(D̂) * b̂_i + log(det(D̂)) + log(det(R̂_i)) $$

In [ ]:
# 단일 클러스터(또는 관측값)에 대한 Negative Log Likelihood 계산
def nll_i(y_i, f_hat_i, Z_i, b_hat_i, R_hat_i, D_hat):
    # y_i: 클러스터에서의 관측 값
    # f_hat_i: 클러스터에서의 예측 값
    # Z_i: 클러스터의 feature map
    # b_hat_i: estimated random effects for the cluster
    # R_hat_i: estimated residual error covariance matrix for the cluster
    # D_hat: estimated random effects covariance matrix
    return np.transpose(y_i - f_hat_i - Z_i @ b_hat_i) @ np.linalg.inv(R_hat_i) @ (y_i - f_hat_i - Z_i @ b_hat_i) + \
                        b_hat_i @ np.linalg.inv(D_hat) @ b_hat_i + np.log(np.linalg.det(D_hat)) + np.log(np.linalg.det(R_hat_i))

In [ ]:
# 전체 데이터셋에서 Negative Log Likelihood 계산 함수
def compute_nll(model, X, y, b_hat, D_hat, sig2e_est, maps2ind, n_clusters, cnt_clusters):
    # model : Mixed Effect Neural Network의 DNN 모형
    # X : 입력 변수
    # y : 타겟 변수
    # b_hat : 모든 클러스터에 대한 추정된 random effects
    # D_hat : 추정된 random effects 공분산 행렬
    # sig2e_est : 추정된 잔차 오차 분산
    # maps2ind : 클러스터 인덱스에서 데이터 인덱스로의 매핑 정보
    # n_clusters : 클러스터의 개수
    # cnt_clusters : 클러스터별 데이터 포인트의 수
    f_hat = model.predict(X, verbose=0).reshape(X.shape[0]) # 모든 데이터 포인트에 대한 fixed effects 예측
    Z = get_feature_map(model, X) # 모든 데이터에 대하여 Feature map 계산
    nll = 0 # Negative log likelihood 초기화
    for cluster_id in range(n_clusters): # 각 클러스터에 대하여 반복수행 구문
        indices_i = maps2ind[cluster_id] # 현재 클러스터의 데이터 포인트 인덱스 호출
        n_i = cnt_clusters[cluster_id] # 현재 클러스터의 데이터 포인트 개수 호출
        y_i = y[indices_i] # 현재 클러스터에 대한 타겟변수 호출
        Z_i = Z[indices_i, :] # 현재 클러스터에 대한 feature map 호출
        I_i = np.eye(n_i) # n_i 크기의 항등행렬
        f_hat_i = f_hat[indices_i] # 현재 클러스터에 대한 예측값 호출
        R_hat_i = sig2e_est * I_i # 현재 클러스터에 대한 추정된 잔차 오차 공분산 행렬 계산
        b_hat_i = b_hat[cluster_id, :] # 현재 클러스터에 대한 추정된 random effects 호출
        nll = nll + nll_i(y_i, f_hat_i, Z_i, b_hat_i, R_hat_i, D_hat) # 현재 클러스터에 대한 Negative Log Likelihood 계산 후 총합 더하기
    return nll

In [ ]:
# 모형 훈련 중에 조기 종료 조건 확인 함수
def check_stop_model(nll_valid, best_loss, wait_loss, patience):
    # nll_valid : 현재의 검증 손실 값(Negative Log likelihood)
    # best_loss : 이전 단계까지의 가장 낮은 검증 손실 값
    # wait_loss : 검증 손실이 개선되지 않은 연속적인 에포크 수
    # patience : 모형의 검증 손실이 개선되지 않아도 허용하는 최대 에포크 수
    # 반환
    ## best_loss : 현재 검증 손실이 이전의 최고 손실보다 낮다면 현재의 검증 손실로 업데이트하고, 그렇지 않다면 이전의 최고 손실을 그대로 유지
    ## wait_loss : 현재 검증 손실이 이전의 최고 손실보다 낮다면 0으로 초기화하고, 그렇지 않다면 wait_loss를 1 증가
    # stop_model : 검증 손실이 개선되지 않은 에포크 수가 patience를 초과하면 True가 되어 모형 훈련을 조기에 종료
    stop_model = False
    if nll_valid < best_loss:
        best_loss = nll_valid
        wait_loss = 0
    else:
        wait_loss += 1
        if wait_loss >= patience:
            stop_model = True
    return best_loss, wait_loss, stop_model

### EM Algorithm

1. Initialize the neural network Γ(0) and βˆ(0)
2. Initialize  uˆ_i(0) = 0_q+1, σˆ2_(0) = 1, and Σˆ_u(0) = I
3. E-Step : update the fixed part response y^fixed_i(k)

    $$ by y^{fixed}_{i(k)} = y_{i} - {\Gamma}_{i(k-1)}uˆ_{i(k-1)} $$
    $$ where {\Gamma}_{i(k-1)} = {\Gamma}_{(k-1)}(X_{i}) $$
4. Obtain Γ_(k) and β_(k) by training neural networks with data by solving the minimization problem
    $$ ({\Gamma}(k), {\beta}(k)) = argmin_{({\Gamma}, {\beta})} Σ^{n}_{i=1} || y^{fixed}_{i(k)} − {\Gamma}(X_{i}){\beta}||^{2} $$
5. Update random effects uˆ_k and residuals εˆ_i(k) by
    $$ u^{ˆ}_{i(k)} ={\Sigma}^{ˆ}_{u(k-1)} {\Gamma}^{T}_{i(k)}V^{ˆ}_{i(k-1)}(y_{i}−{\Gamma}_{i(k)}{\beta}^{ˆ}_{(k)}) $$
    $$ {\epsilon}^{ˆ}_{i(k)} = y_{i} - {\Gamma}_{i(k)}{\beta}^{ˆ}_{(k)} - {\Gamma}_{i(k)}u^{ˆ}_{(k)} $$
    where $$ V_{i(k-1)} = {\Sigma}^{ˆ}_{u(k-1)}{\Gamma}^{T}_{i(k)} + {\sigma}^{ˆ}_{(k-1)}I_{n} $$
    which follow from $$ {E[u^{ˆ}_{i}|y_{i}]} = {\Sigma}^{ˆ}_{u}{\Gamma}^{T}_{i}{V^{ˆ}_{i}(y_{i} - {\Gamma}_{i}{\beta}^{ˆ})} $$
    $$ {E[{\epsilon}^{ˆ}_{i}|y_{i}]} = {\sigma}^{2}V^{ˆ-1}_{i}(y_{i} - {\Gamma}_{i}{\beta}^{ˆ}) $$
    $$ = y_{i} - {\Gamma}_{i}{\beta}^{ˆ} - {\Gamma}_{i}{E[u^{ˆ}_{i}|y_{i}]} $$
6. M-Step : update σˆ^2_(k) and Σˆ_u(k) by
    $$ {\sigma}^{ˆ2}_{(k)} = {1/n}{\Sigma}^{N}_{i=1}(||{\epsilon}^{ˆ}_{i(k)}||^{2} + {\sigma}^{ˆ2}_{(k-1)}(n_{i} - {\sigma}^{ˆ2}_{(k-1)}tr(V^{ˆ}_{i(k-1)}))), $$
    $$ {\Sigma}^{ˆ}_{u(k)} = {1/N}{\Sigma}^{N}_{i=1}(u^{ˆ}_{i(k)}u^{ˆT}_{i(k)} + {\Sigma}^{ˆ}_{u(k-1)} - {\Sigma}^{ˆ}_{u(k-1)}{\Gamma}^{T}_{u(k-1)}V^{ˆ-1}_{i(k-1)}{\Gamma}_{i(k)}{\Sigma}^{ˆ}_{u(k-1)}) $$
    when n = Σ^N_i=1 n_i, which follow from
    $$ {E[{\sigma}^{ˆ}|y]} = 1/n {\Sigma}^{N}_{i=1}(||{\epsilon}^{ˆ}_{i}||^{2} + {\sigma}^{2}(n_{i} - {\sigma}^{2}tr(V^{-1}))) $$
    $$ {E[{\Sigma}^{ˆ}|y]} = 1/N{\Sigma}^{N}_{i=1}(u^{ˆ}_{i}u^{ˆT}_{i} + {\Sigma}_{u} - {\Sigma}_{u}{\Gamma}^{T}_{i}V^{-1}_{i}{\Gamma}_{i}{\Sigma}_u) $$
    where $$ {\sigma}^{ˆ2} = 1/n{\Sigma}^{N}_{i=1}||{\epsilon}_{i}||^{2} $$
    and $$ {\Sigma}^{ˆ}_{u} = 1/N{\Sigma}^{N}_{i=1}u_{i}u^{T}_{i} $$
    and MLEs for σ^2 and Σ_u, respectively

In [ ]:
def menet_fit(model, X, y, clusters, n_clusters, batch_size, epochs, patience, verbose=False):
    # 데이터 세트 분할
    X_train, X_valid, y_train, y_valid, clusters_train, clusters_valid = train_test_split(X, y, clusters, test_size=0.1, random_state=0)

    # 각 클러스터에 속한 인덱스를 리스트로 만듦
    maps2ind_train = [list(np.where(clusters_train == i)[0]) for i in range(n_clusters)]
    maps2ind_valid = [list(np.where(clusters_valid == i)[0]) for i in range(n_clusters)]
    
    # 각 클러스터에 속한 샘플 수를 카운트
    cnt_clusters_train = Counter(clusters_train)
    cnt_clusters_valid = Counter(clusters_valid)
    
    # 모델읠 Feature map을 획득
    Z = get_feature_map(model, X_train)
    
    d = Z.shape[1] # Feature map의 차원 수
    b_hat = np.zeros((n_clusters, d)) # b_hat 초기화
    D_hat = np.eye(d) # D_hat 초기화(단위 행렬)
    D_hat_list = []
    sig2e_est = 1.0 # 초기의 sig2e 추정값
    
    # 각 데이터 세트의 Negative Log Likelihood(NLL)을 기록하는 dictionary
    nll_history = {'train': [], 'valid': []}
    best_loss = np.inf # 최고의 손실값을 기록하는 변수, 초기값은 무한대
    wait_loss = 0 # 얼마나 오래 기다려야 하는지 판단하는 변수
    
    for epoch in range(epochs):
        y_star = np.zeros(y_train.shape) # y_star 초기화
        # E - Step
        # 각 클러스터에 대하여
        for cluster_id in range(n_clusters):
            indices_i = maps2ind_train[cluster_id] # 해당 클러스터에 속하는 샘플의 인덱스
            b_hat_i = b_hat[cluster_id] # 해당 클러스터의 b_hat
            y_star_i = y_train[indices_i] - Z[indices_i] @ b_hat_i # y_star 계산
            y_star[indices_i] = y_star_i # y_star 갱신
            
        # 모델을 y_star 에 대하여 학습
        model.fit(X_train, y_star, batch_size = batch_size, epochs=1, verbose=0)
        
        Z = get_feature_map(model, X_train) # 모델의 feature map 갱신
        
        f_hat = model.predict(X_train, verbose=0).reshape(X_train.shape[0]) # 모델의 예측값
        sig2e_est_sum = 0 # sig2e 추정값의 합을 초기화
        D_hat_sum = 0 # D_hat의 합을 초기화
        
        # M - Step
        # 각 클러스터에 대하여
        for cluster_id in range(n_clusters):
            indices_i = maps2ind_train[cluster_id] # 해당 클러스터에 속하는 샘플의 인덱스
            n_i = cnt_clusters_train[cluster_id] # 해당 클러스터에 속하는 샘플의 수
            f_hat_i = f_hat[indices_i] # 해당 클러스터에 속하는 샘플의 예측값
            y_i = y_train[indices_i] # 해당 클러스터에 속하는 샘플의 실제값
            Z_i = Z[indices_i, :] # 해당 클러스터에 속하는 샘플의 feature map
            
            # V_hat 계산
            V_hat_i = Z_i @ D_hat @ np.transpose(Z_i) + sig2e_est * np.eye(n_i)
            V_hat_inv_i =  np.linalg.inv(V_hat_i) # V_hat의 역행렬
            b_hat_i = D_hat @ np.transpose(Z_i) @ V_hat_inv_i @ (y_i - f_hat_i)# b_hat 갱신
            eps_hat_i = y_i - f_hat_i - Z_i @ b_hat_i # 잔차 계산
            b_hat[cluster_id, :] = b_hat_i.squeeze() # b_hat 갱신
            # sig2e 추정값의 합 갱신
            sig2e_est_sum = sig2e_est_sum + np.transpose(eps_hat_i) @ eps_hat_i + sig2e_est * (n_i - sig2e_est * np.trace(V_hat_inv_i))
            # D_hat의 합 갱신
            D_hat_sum = D_hat_sum + b_hat_i @ np.transpose(b_hat_i) + (D_hat - D_hat @ np.transpose(Z_i) @ V_hat_inv_i @ Z_i @ D_hat)
            
        sig2e_est = sig2e_est_sum / X_train.shape[0] # sig2e 추정값 갱신
        D_hat = D_hat_sum / n_clusters # D_hat 계산
        D_hat_list.append(D_hat)
        
        # NLL 계산
        # nll_train = compute_nll(model, X_train, y_train, b_hat, D_hat, sig2e_est, maps2ind_train, n_clusters, cnt_clusters_train)
        nll_valid = compute_nll(model, X_valid, y_valid, b_hat, D_hat, sig2e_est, maps2ind_valid, n_clusters, cnt_clusters_valid)
        # nll_history['train'].append(nll_train)
        nll_history['valid'].append(nll_valid)
        best_loss, wait_loss, stop_model = check_stop_model(nll_valid, best_loss, wait_loss, patience)
        
        # 현재 상태 출력
        if verbose:
            print(f'epoch: {epoch}, val_loss: {nll_valid:.2f}, sig2e_est: {sig2e_est:.2f}')
        # 모델을 멈출 경우, 반복문 종료
        if stop_model:
            break
        
    n_epochs = len(nll_history['valid']) # 총 epoch 수
    return model, b_hat, sig2e_est, n_epochs, nll_history, D_hat_list

# Model Training


In [ ]:
q = int(max([au_import['z0'].max()])+ 1)

train, test = train_test_split(au_import, test_size=0.2, random_state=530)

X_cols = list(au_import.columns)
X_cols.remove('y')
y_cols = ['y']

X_train = train[X_cols].values
y_train = train[y_cols].values.squeeze()
clusters_train = train['z0'].values
# clusters_train = clusters_train.reshape(clusters_train.shape[0],1)

X_test = test[X_cols].values
y_test = test[y_cols].values.squeeze()
clusters_test = test['z0'].values
# clusters_test = clusters_test.reshape(clusters_test.shape[0],1)

print(X_train.shape, y_train.shape, clusters_train.shape)
print(X_test.shape, y_test.shape, clusters_test.shape)

In [ ]:
model = get_model()

batch_size = 32
epochs = 10
patience = 5
verbose=True

## Menet fit


In [ ]:
menet_model, b_hat, sig2e_est, n_epochs, nll_history, D_hat_list = menet_fit(model, X_train, y_train, clusters_train, q, batch_size, epochs, patience, verbose)

In [ ]:
# Check D_hat_list
print(len(D_hat_list))
print(np.array(D_hat_list).shape)
print(D_hat_list[0][0])

In [ ]:
D_hat_list[9]

# MeNet Predict


In [ ]:
def menet_predict(model, X, clusters, n_clusters, b_hat):
    y_hat = model.predict(X, verbose=0).reshape(X.shape[0])
    Z = get_feature_map(model, X)
    for cluster_id in range(n_clusters):
        indices_i = np.where(clusters == cluster_id)[0]
        if len(indices_i) == 0:
            continue
        b_i = b_hat[cluster_id, :]
        Z_i = Z[indices_i, :]
        y_hat[indices_i] = y_hat[indices_i] + Z_i @ b_i
    return y_hat

In [ ]:
y_pred = menet_predict(model, X_test, clusters_test, q, b_hat)

In [ ]:
y_pred[:5]

In [ ]:
y_test[:5]

In [ ]:
mse = np.mean((y_pred - y_test)**2)
rmse = np.sqrt(mse)

In [ ]:
print(mse)
print(rmse)